<a href="https://colab.research.google.com/github/kaushikbar/spark/blob/master/03_04/Working_with_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with columns

## Download and install Spark

In [1]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [0]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

## Setup environment

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [0]:
#!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
#!ls -l

In [0]:
#!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
#!ls -l

In [3]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
rc.show(5)

+-------+-----------+-------------------+--------------------+----+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block|IUCR|     Primary Type|      Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+----+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|4968078|   HM580684|2006-09-04 09:32:56| 005XX S FRANKLIN ST|1350|CRIMINAL TRESPASS|TO STATE SUP LAND|               OTHER|  true|   false|0131|   

## Working with columns

**Display only the first 5 rows of the column name IUCR **

In [5]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|1350|
|0850|
|0460|
|0890|
|0810|
+----+
only showing top 5 rows



In [6]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|1350|
|0850|
|0460|
|0890|
|0810|
+----+
only showing top 5 rows



In [7]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|1350|
|0850|
|0460|
|0890|
|0810|
+----+
only showing top 5 rows



In [9]:
rc[['IUCR']].show(5)

+----+
|IUCR|
+----+
|1350|
|0850|
|0460|
|0890|
|0810|
+----+
only showing top 5 rows



  **Display only the first 4 rows of the column names Case Number, Date and Arrest**

In [10]:
rc[['Case Number', 'Date', 'Arrest']].show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HM580684|2006-09-04 09:32:56|  true|
|   HM577323|2006-08-31 20:30:00| false|
|   HM579928|2006-09-03 04:30:00| false|
|   HM580630|2006-09-03 22:30:00| false|
+-----------+-------------------+------+
only showing top 4 rows



In [11]:
rc.select(['Case Number', 'Date', 'Arrest']).show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HM580684|2006-09-04 09:32:56|  true|
|   HM577323|2006-08-31 20:30:00| false|
|   HM579928|2006-09-03 04:30:00| false|
|   HM580630|2006-09-03 22:30:00| false|
+-----------+-------------------+------+
only showing top 4 rows



In [12]:
rc.select('Case Number', 'Date', 'Arrest').show(4)

+-----------+-------------------+------+
|Case Number|               Date|Arrest|
+-----------+-------------------+------+
|   HM580684|2006-09-04 09:32:56|  true|
|   HM577323|2006-08-31 20:30:00| false|
|   HM579928|2006-09-03 04:30:00| false|
|   HM580630|2006-09-03 22:30:00| false|
+-----------+-------------------+------+
only showing top 4 rows



** Add a column with name One, with entries all 1s **

In [0]:
from pyspark.sql.functions import lit

In [14]:
rc.withColumn('One', lit(1)).show(5)

+-------+-----------+-------------------+--------------------+----+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|     ID|Case Number|               Date|               Block|IUCR|     Primary Type|      Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+-------+-----------+-------------------+--------------------+----+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|4968078|   HM580684|2006-09-04 09:32:56| 005XX S FRANKLIN ST|1350|CRIMINAL TRESPASS|TO STATE SUP LAND|               OTHER|  true|   fa

** Remove the column IUCR **

In [15]:
rc = rc.drop('IUCR')
rc.show(5)

+-------+-----------+-------------------+--------------------+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|     ID|Case Number|               Date|               Block|     Primary Type|      Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+-------+-----------+-------------------+--------------------+-----------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|4968078|   HM580684|2006-09-04 09:32:56| 005XX S FRANKLIN ST|CRIMINAL TRESPASS|TO STATE SUP LAND|               OTHER|  true|   false|0131|     001|   2|         